## MolSSI Molecular Mechanics Interoperable Components
### Conceptual Mockup

This Jupyter Notebook is a conceptual mockup of the MolSSI MM Interoperable Components. The goal of this mockup is to demonstrate the ideas and goals of the components and get feedback on possible design choices.
We present an example Molecular Mechanics pipeline:
 - System Comstruction
 - Minimization and Equilibration
 - Production Run
 - Analysis

For each stage in the pipeline, we have a number of available components, using different codes or operations, that can fill the same role.
Though using different codes and potentially performing different operations, the components that can be used in each stage all operate on the same input parameters and, with the exception of specific analysis operations, provide the same output parameters.

By defining the I/O as a contract between stages, we create a system pipeline that is very modular and easy to adjust between components. More specifically, any component that adheres to the I/O specification of a stage can be used in place of any other component for that stage. The correctness of a specific pipeline, with components in place, is dependent on each component adhering to the I/O specification as well as it performing operations that  are applicable to a stage. {{{need an example here?}}}



### Available Components
We first import the available components for the mockup. Only one component from each stage is necessary to execute the pipeline, but we import all here for clarity.

In [1]:
import os
from mminterconcept.construction import (protein_constructor,
                           protein_constructor_vacuum,
                           protein_constructor_solvate,
                           protein_constructor_solvate_ion)

from mminterconcept.min_eq import (min_eq,
                     gro_minimize,
                     gro_eq_nvt,
                     gro_eq_npt,
                     omm_minimize,
                     omm_eq)

from mminterconcept.production import (production,
                         gromacs_production,
                         openmm_production)

from mminterconcept.analysis import (analysis,
                       mda_com,
                       mda_density,
                       mda_rdf,
                       mda_rmsd,
                       mda_rog,
                       mdtraj_com,
                       mdtraj_density,
                       mdtraj_rdf,
                       mdtraj_rmsd,
                       mdtraj_rog)

### Input processing
Here we start with a PDB file of a system for use in our pipeline.

In [4]:

#mdp = os.path.join(os.path.realpath(__file__), "data", "em.mdp")
mdp = os.path.join(os.getcwd(), os.path.join("mminterconcept", "data", "em.mdp"))


### System Construction
This stage constructs the system from the given pdbID and mdp file.

#### API-Schema
```python
protein_constructor(pdbID, mdp, **kwargs) -> Tuple[mdtraj.Trajectory, str]
```

All components for this stage adhere to the above API-Schema. They take as input a pdbID and an mdp and return a tuple containing an mdtraj style Trajectory and a string containing the topology.

In [5]:
trajectory, topology = protein_constructor_vacuum(pdbID, mdp)

### Minimization and Equilibration

#### API-Schema
```python
min_eq(*, trajectory: mdtraj.Trajectory, topology: str) -> Tuple[mdtraj.Trajectory, str]
```
All components for this stage adhere to the above API-Schema. They take as input an mdtraj style Trajectory, and a topology as a string. They return a tuple containing an mdtraj style Trajectory and a topology as a string.

In [6]:
trajectory, topology = omm_minimize(trajectory=trajectory, topology=topology)

In [7]:
trajectory, topology = omm_eq(trajectory=trajectory, topology=topology)

### Production Run

#### API-Schema
```python
production(*, trajectory: mdtraj.Trajectory, topology: str) -> mdtraj.Trajectory
```
All components for this stage adhere to the above API-Schema. They take as input an mdtraj style Trajectory, and a topology as a string. They return an mdtraj style Trajectory.

In [8]:
trajectory, topology = openmm_production(trajectory=trajectory, topology=topology)

### System Analysis

In [4]:
import nglview
from mminterconcept.construction.tools import ImportPDB 

#pdbID = '1LFH' # LACTOFERRIN globular glycoprotein ~ 10549 atoms
pdbID = '1L2Y' # ARTIFICIAL Trp-Cage miniprotein ~ 304 atoms 

with ImportPDB(pdbID) as traj:
    view = nglview.show_mdtraj(traj)

view

NGLWidget(max_frame=37)

In [ ]:
import os
import mdtraj
from mminterconcept.analysis import mdtraj_com, mdtraj_rmsd, mdtraj_rog, mdtraj_rdf, mdtraj_den
from mminterconcept.analysis import mda_com, mda_rmsd, mda_rog, mda_rdf, mda_den

# This is for testing purposes ... the 5 lines below should be eventually deleted
path_to_files = os.path.join('mminterconcept', 'data', '1LFH', 'gmx')
struct = os.path.join(path_to_files, 'struct', 'system_ionized.pdb')
traj = os.path.join(path_to_files, 'traj', 'MD-1ps.trr')

topol, trajectory = mdtraj.load(struct), mdtraj.load(traj, top=struct)

###################################################
####### Compute phys variables with MDTraj ########
###################################################

# Temporal analyses
time_mdt, com_mdt = mdtraj_com(topol, trajectory)
time_mdt, den_mdt = mdtraj_den(topol, trajectory)
time_mdt, rmsd_mdt = mdtraj_rmsd(topol, trajectory)
time_mdt, rog_mdt = mdtraj_rog(topol, trajectory)

# Spatial (structural) analysis
rdist_mdt, rdf_mdt = mdtraj_rdf(topol, trajectory)


###################################################
###### Compute phys variables with MDAnalysis #####
###################################################

# Temporal analyses
time_mda, com_mda = mda_com(topol, trajectory)
time_mda, den_mda = mda_den(topol, trajectory)
time_mda, rmsd_mda = mda_rmsd(topol, trajectory)
time_mda, rog_mda = mda_rog(topol, trajectory)

# Spatial (structural) analysis
rdist_mda, rdf_mda = mda_rdf(topol, trajectory)

AttributeError: 'str' object has no attribute 'save'

In [ ]:
# Plot the x-component of the protein center of mass a function of time
from mminterconcept.analysis import plot_com

plot_com(time_mdt, com_mdt[:,0], time_mda, com_mda[:,0])

In [ ]:
# Plot root-mean-square deviation as a function of time
from mminterconcept.analysis import plot_rmsd

plot_rmsd(time_mdt, rmsd_mdt, time_mda, rmsd_mda)

In [ ]:
# Plot mass density of the system as a function of time
import matplotlib
%matplotlib inline

plot_den(time_mdt, den_mdt, time_mda, den_mda)

In [ ]:
from mminterconcept.analysis import plot_den

plot_den(time_mdt, den_mdt, time_mda, den_mda)
